# Field Simulation and Visualization

This notebook demonstrates the core simulation capabilities of MyceliumFractalNet, including:
- Turing morphogenesis patterns
- Nernst potential computation
- Field evolution visualization
- Growth event tracking

In [ ]:
# Install required packages if running in Colab
import sys

if "google.colab" in sys.modules:
    !pip install -q mycelium-fractal-net matplotlib numpy

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib.animation import FuncAnimation

from mycelium_fractal_net import (
    compute_nernst_potential,
    make_simulation_config_demo,
    run_mycelium_simulation_with_history,
)

print("✓ Imports successful")

## 1. Basic Field Simulation

Let's start with a simple simulation using default parameters.

In [ ]:
# Create a simulation configuration
config = make_simulation_config_demo()

print("Simulation Configuration:")
print(f"  Grid size: {config.grid_size}x{config.grid_size}")
print(f"  Steps: {config.steps}")
print(f"  Alpha (diffusion): {config.alpha}")
print(f"  Turing enabled: {config.turing_enabled}")

In [ ]:
# Run simulation with history tracking
result = run_mycelium_simulation_with_history(config)

print("\nSimulation Results:")
print(f"  Final field shape: {result.field_final.shape}")
print(f"  Field history shape: {result.field_history.shape}")
print(f"  Growth events: {result.growth_events}")
print(f"  Turing activations: {result.turing_activations}")
print(f"  Clamping events: {result.clamping_events}")
print(f"  Potential range: [{result.field_final.min():.1f}, {result.field_final.max():.1f}] mV")

## 2. Visualize Field State

Visualize the final field state as a heatmap showing membrane potentials.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Initial state
im1 = axes[0].imshow(result.field_history[0], cmap="viridis", vmin=-100, vmax=50)
axes[0].set_title("Initial State (t=0)")
axes[0].set_xlabel("X")
axes[0].set_ylabel("Y")
plt.colorbar(im1, ax=axes[0], label="Potential (mV)")

# Mid state
mid_idx = len(result.field_history) // 2
im2 = axes[1].imshow(result.field_history[mid_idx], cmap="viridis", vmin=-100, vmax=50)
axes[1].set_title(f"Mid State (t={mid_idx})")
axes[1].set_xlabel("X")
axes[1].set_ylabel("Y")
plt.colorbar(im2, ax=axes[1], label="Potential (mV)")

# Final state
im3 = axes[2].imshow(result.field_final, cmap="viridis", vmin=-100, vmax=50)
axes[2].set_title(f"Final State (t={len(result.field_history) - 1})")
axes[2].set_xlabel("X")
axes[2].set_ylabel("Y")
plt.colorbar(im3, ax=axes[2], label="Potential (mV)")

plt.tight_layout()
plt.show()

print("Field evolution shows Turing morphogenesis patterns emerging over time.")

## 3. Animated Field Evolution

Create an animation showing how the field evolves over time.

In [ ]:
# Create animation
fig, ax = plt.subplots(figsize=(8, 6))

# Initialize with first frame
im = ax.imshow(result.field_history[0], cmap="viridis", vmin=-100, vmax=50, animated=True)
ax.set_xlabel("X")
ax.set_ylabel("Y")
title = ax.set_title("Field Evolution: t=0")
plt.colorbar(im, ax=ax, label="Potential (mV)")


def update(frame):
    im.set_array(result.field_history[frame])
    title.set_text(f"Field Evolution: t={frame}")
    return [im, title]


# Create animation (sample every 5th frame for speed)
frames = range(0, len(result.field_history), 5)
anim = FuncAnimation(fig, update, frames=frames, interval=100, blit=True)

# Display in notebook
plt.close()  # Don't show static image
HTML(anim.to_jshtml())

## 4. Nernst Potential Computation

Demonstrate the Nernst equation for ion equilibrium potentials.

In [ ]:
# Compute Nernst potential for K+ ion
E_K = compute_nernst_potential(
    z_valence=1,
    concentration_out_molar=5e-3,  # [K+]out = 5 mM
    concentration_in_molar=140e-3,  # [K+]in = 140 mM
    temperature_k=310.0,  # 37°C body temperature
)

print(f"Nernst Potential for K+: {E_K * 1000:.2f} mV")
print("Expected value: -89.01 mV")
print(f"Match: {abs(E_K * 1000 - (-89.01)) < 0.1}")

# Compute for different ions
ions = [
    ("K+", 1, 5e-3, 140e-3),
    ("Na+", 1, 145e-3, 12e-3),
    ("Ca2+", 2, 2e-3, 0.0001e-3),
    ("Cl-", -1, 110e-3, 4e-3),
]

print("\nEquilibrium Potentials for Common Ions:")
for name, z, c_out, c_in in ions:
    E = compute_nernst_potential(z, c_out, c_in, 310.0)
    print(f"  {name:4s}: {E * 1000:7.2f} mV")

## 5. Statistical Analysis

Analyze the distribution of membrane potentials across the field.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Histogram of final potentials
axes[0].hist(result.field_final.flatten(), bins=50, alpha=0.7, edgecolor="black")
axes[0].axvline(
    result.field_final.mean(),
    color="red",
    linestyle="--",
    label=f"Mean: {result.field_final.mean():.1f} mV",
)
axes[0].axvline(-70, color="green", linestyle="--", label="Resting potential: -70 mV")
axes[0].set_xlabel("Membrane Potential (mV)")
axes[0].set_ylabel("Frequency")
axes[0].set_title("Distribution of Membrane Potentials")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Time evolution of mean potential
mean_potentials = [frame.mean() for frame in result.field_history]
axes[1].plot(mean_potentials, linewidth=2)
axes[1].set_xlabel("Time Step")
axes[1].set_ylabel("Mean Potential (mV)")
axes[1].set_title("Evolution of Mean Membrane Potential")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nStatistics:")
print(f"  Mean potential: {result.field_final.mean():.2f} mV")
print(f"  Std deviation: {result.field_final.std():.2f} mV")
print(f"  Min potential: {result.field_final.min():.2f} mV")
print(f"  Max potential: {result.field_final.max():.2f} mV")

## 6. Parameter Exploration

Explore how different parameters affect the simulation.

In [ ]:
# Compare different alpha values (diffusion rate)
alphas = [0.1, 0.3, 0.5, 0.7]
fig, axes = plt.subplots(1, len(alphas), figsize=(16, 4))

for i, alpha in enumerate(alphas):
    config_test = make_simulation_config_demo()
    config_test.alpha = alpha
    config_test.steps = 50  # Shorter simulation
    result_test = run_mycelium_simulation_with_history(config_test)

    im = axes[i].imshow(result_test.field_final, cmap="viridis", vmin=-100, vmax=50)
    axes[i].set_title(f"α = {alpha}\nGrowth events: {result_test.growth_events}")
    axes[i].set_xlabel("X")
    if i == 0:
        axes[i].set_ylabel("Y")
    plt.colorbar(im, ax=axes[i], label="Potential (mV)")

plt.suptitle("Effect of Diffusion Rate (α) on Field Patterns", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("Higher diffusion rates (α) lead to smoother, more uniform fields.")
print("Lower diffusion rates preserve local variations and create more complex patterns.")

## Summary

This notebook demonstrated:
- ✓ Basic field simulation with Turing morphogenesis
- ✓ Visualization of field states and evolution
- ✓ Animated field dynamics
- ✓ Nernst potential computations
- ✓ Statistical analysis of membrane potentials
- ✓ Parameter exploration

For more advanced features, see:
- `02_feature_analysis.ipynb` - Fractal feature extraction
- `03_fractal_exploration.ipynb` - Fractal dimension analysis